# Download model variables

Download as raster layers covering your study area envelope, including:

- At least one **soil** variable from the [POLARIS dataset](http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/) (**find thresholds**)
    - <https://scholars.duke.edu/publication/1381493>
    - <https://gee-community-catalog.org/projects/polaris/>
    - NW Chaney et al. (2019) POLARIS Soil Properties: 30-m Probabilistic Maps of Soil Properties Over the Contiguous United States. [DOI: 10.1029/2018WR022797](https://doi.org/10.1029/2018WR022797)
    - [Accessing and formatting URLS for POLARIS data](https://cuboulder.zoom.us/rec/share/imhCGJcrCgSoE1cJjg02r86GMNjiRz0jwVMJ5c0uWNwBCD5D_0kLSl3CaqLdDI2a.ucGta1EEAiirDUop)
    - [Looping through multiple tiles of POLARIS data](https://cuboulder.zoom.us/rec/share/3S7pGOSv7jztZTvg4RSXLj-GicnHwusIDIEDoETZbUN7ivkc6Ryi5GAJyX9Ly6h2.VXl6iFmLChjjUyOg)
- Elevation from the SRTM (available from the [earthaccess API](https://github.com/nsidc/earthaccess/))
- At least one **climate** variable from the [MACAv2 THREDDS data server](http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_macav2_catalog2.html).
    - Pick 2 of these
    - [GeoNetwork](https://docs.geonetwork-opensource.org/4.2/):
[THREDDS Harvesting](https://docs.geonetwork-opensource.org/4.2/user-guide/harvesting/harvesting-thredds/)

## POLARIS Dataset

Considering `sand` percentage `mean`. POLARIS data are available at 6 depths, and Bluestem has roots down to 5 feet (150 cm), which is the lowest strata measured (100-200 cm). Data in the
[sand 100-200 cm directory](http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/sand/mean/100_200/)
are saved as separate tif files by longitude.
Buvvalo Gap National Grassland is at (centroid) 43.4375° N, 103.0505° W, while Oglala National Grassland is at 42.9404° N, 103.5900° W. That probably suggests

- [lat4344_lon-103-102.tif](http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/sand/mean/100_200/lat4344_lon-103-102.tif) (33M)
- [lat4344_lon-104-103.tif](http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/sand/mean/100_200/lat4344_lon-104-103.tif) (33M)

https://www.gbif.org/occurrence/download/0014700-241007104925546

In [2]:
!conda install gdal

Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package pyogrio-0.10.0-py311hc9b973e_0 requires libgdal-core >=3.9.2,<3.10.0a0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ gdal is installable with the potential options
│  ├─ gdal [3.0.2|3.4.1|3.6.0|3.6.2] would require
│  │  └─ python >=3.10,<3.11.0a0 , which can be installed;
│  ├─ gdal [3.0.2|3.4.1|3.6.0|3.6.2] would require
│  │  └─ python >=3.8,<3.9.0a0 , which can be installed;
│  ├─ gdal [3.0.2|3.4.1|3.6.0|3.6.2] would require
│  │  └─ python >=3.9,<3.10.0a0 , which can be installed;
│  ├─ gdal 3.6.0 would require
│  │  └─ libgdal 3.6.0 h50c47ba_0, which can be installed;
│  ├─ gdal 3.6.2 would require
│  │  └─ libgdal 3.6.2 h50c47ba_1, which can be installed;
│  ├─ gdal 3.6.2 would require
│  │  └─ libgdal 

In [1]:
from osgeo import gdal
import numpy as np

def read_geotiff(filename):
    """Reads a GeoTIFF file and returns a NumPy array."""
    ds = gdal.Open(filename)
    band = ds.GetRasterBand(1)  # Assuming single-band GeoTIFF
    array = band.ReadAsArray()
    return array


ModuleNotFoundError: No module named 'osgeo'

In [ ]:

# Example usage
filename = "data/lat4344_lon-103-102.tif"
data = read_geotiff(filename)

print(data.shape)  # Print the shape of the NumPy array